In [1]:
#please run these commands for first time and comment them out
!pip3 install fuzzywuzzy


You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from fuzzywuzzy import process
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#Reading the csv file
df= pd.read_csv("data.txt")


In [4]:
#dataframe processing

#dropping nan values
df = df.dropna()   

#dropping index 0
df.pop(df.columns[0])


In [19]:
# creating a Pivot matrix in which all the features will be user id and rows will be movies

#eif user doesn't rated any movie it will be assigned NAN value so we will replace that with 0
## First lets create a Pivot matrix
#,
movie_features_df=cleaned_df.pivot_table(index='program_desc',columns='user_id',values='rating',aggfunc='first').fillna(0)
movie_features_df

user_id,10053,10154,10234,10257,10316,10335,1035,10438,10461,10522,...,938,9420,9481,9540,9559,9601,9683,9706,9784,9988
program_desc,,,,,,,,,,,,,,,,,,,,,
#FollowFriday (Thriller-MOVIE),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10 Days in a Madhouse (Thriller-MOVIE),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100 treets (Drama-MOVIE),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101 Dalmatians (Family-MOVIE),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"2 Fil-Isaaf (T),Season 4,Episode Comedy (Comedy-SERIES/EPISODES)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zombieland (Comedy-MOVIE),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Zoo,Season 1,Episode Action (Action-SERIES/EPISODES)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Zoo,Season 3,Episode Action (Action-SERIES/EPISODES)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
#converting pivot matrixs into array matrix using scipy.sparse library
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df.astype(float))

from sklearn.neighbors import NearestNeighbors
#by training unsupervised KNN, we will calculate the cosine similarity between recommendations
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [23]:
def content_based_filtering(movie_name,df):
    #preprocessing import columns
    try:
        #processing dataframe
    
        # Delete duplicate rows based on specific columns 
        df = df.drop_duplicates(subset=["overview"], keep=False)
        
        #reseting index
        df.reset_index(drop=True, inplace=True)
        df['index'] = df.reset_index().index
        
        #selecting important_features
        selected_features=['program_class','overview','program_genre','original_name']
        for feature in selected_features:
            df[feature]=df[feature].fillna('')

                #combining all features
        combined_features= df['program_genre']+' '+df['overview']+' '+df['program_class']+' '+df['original_name']

                #converting the combined features into numeric values
        vectorizer=TfidfVectorizer()
        feature_vectors=vectorizer.fit_transform(combined_features)

                #Cosine Similarity
        similarity=cosine_similarity(feature_vectors)

        index_of_the_movie=process.extractOne(movie_name,df['overview'])[2]

        #getting list of similar movies
        similarity_score=list(enumerate(similarity[index_of_the_movie]))

        sorted_similar_movies=sorted(similarity_score,key=lambda x:x[1],reverse=True)

        print("Recommended movies for {}".format(movie_name))
        i=0
        for movie in sorted_similar_movies:
            index=movie[0]
            title_from_index=df[df.index==index]['overview'].values[0]
            if i<6:
                print(i,' ',title_from_index)
                i+=1
            else:
                break
    except:
        print('Movie not Found')

In [24]:
def search_movies(movie_name,data,df):
    try:
        query_index=process.extractOne(movie_name,cleaned_df['program_desc'])[2]
        distances, indices = model_knn.kneighbors(data[query_index], n_neighbors = 7)
        print("Recommendation for movie {}".format(movie_name))
        for i in range(0, len(distances.flatten())):
            #printing other movies that have more similarity to it
            print(                 '{0}: {1}, with distance of {2}:'.format(i,movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))
    except:
        content_based_filtering(movie_name,df)
        
    

In [30]:
movie_name=input("Search movie: ")
search_movies(movie_name,movie_features_df_matrix,df)

Recommendation for movie Horro
0: Teenage Mutant Ninja Turtles,Season 14,Episode Animation (Animation-SERIES/EPISODES), with distance of 0.0:
1: Dora the Explorer,Season 2,Episode Animation (Animation-SERIES/EPISODES), with distance of 0.0:
2: Sonic Underground,Season 23,Episode Animation (Animation-SERIES/EPISODES), with distance of 0.0:
3: Sicario (Action-MOVIE), with distance of 1.0:
4: Shrek (Animation-MOVIE), with distance of 1.0:
5: Six,Season 0 (Action-SERIES/EPISODES), with distance of 1.0:
6: Sleight (Action-MOVIE), with distance of 1.0:


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9191ad2a-ea97-4654-a2ba-f4ecb97808a0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>